# Show how constraining is done

In [1]:
import json
import fair
from climateforcing.utils import check_and_download
import numpy as np
import pandas as pd
import scipy.stats as st
from scipy.interpolate import interp1d

import matplotlib.pyplot as pl

## Download pre-prepared files from AR6

In [2]:
# get the random seeds used in AR6
check_and_download("https://raw.githubusercontent.com/chrisroadmap/ar6/main/data_input/random_seeds.json", "../data_input/random_seeds.json")

In [3]:
# get two layer model tunings used in AR6
check_and_download(
    "https://raw.githubusercontent.com/chrisroadmap/ar6/main/data_input/tunings/cmip6_twolayer_tuning_params.json",
    "../data_input/cmip6_twolayer_tuning_params.json"
)

In [4]:
# aerosol tunings to CMIP6 models
check_and_download(
    "https://raw.githubusercontent.com/chrisroadmap/ar6/main/data_input/tunings/cmip6_aerosol_coefficients.json",
    "../data_input/cmip6_aerosol_coefficients.json"
)

In [5]:
# rcmip emissions files
check_and_download(
    'https://rcmip-protocols-au.s3-ap-southeast-2.amazonaws.com/v5.1.0/rcmip-emissions-annual-means-v5-1-0.csv',
    '../data_input_large/rcmip-emissions-annual-means-v5-1-0.csv'
)

## Ensemble generation

We follow similar lines to AR6, but with a smaller ensemble (= time)

In [6]:
with open('../data_input/random_seeds.json', 'r') as filehandle:
    SEEDS = json.load(filehandle)

In [7]:
SAMPLES = 100000
F2XCO2_MEAN = 3.93
F2XCO2_NINETY = 0.47
NINETY_TO_ONESIGMA = st.norm.ppf(0.95)

### Climate response

In [8]:
with open("../data_input/cmip6_twolayer_tuning_params.json", "r") as read_file:
    params = json.load(read_file)
cmip6_models = list(params['q4x']['model_data']['EBM-epsilon'].keys())
cmip6_models
NMODELS = len(cmip6_models)

geoff_data = np.zeros((NMODELS, 6))
for im, model in enumerate(cmip6_models):
    geoff_data[im,0] = params['q4x']['model_data']['EBM-epsilon'][model]
    geoff_data[im,1] = params['lamg']['model_data']['EBM-epsilon'][model]
    geoff_data[im,2] = params['cmix']['model_data']['EBM-epsilon'][model]
    geoff_data[im,3] = params['cdeep']['model_data']['EBM-epsilon'][model]
    geoff_data[im,4] = params['gamma_2l']['model_data']['EBM-epsilon'][model]
    geoff_data[im,5] = params['eff']['model_data']['EBM-epsilon'][model]

geoff_df = pd.DataFrame(geoff_data, columns=['q4x','lamg','cmix','cdeep','gamma_2l','eff'], index=cmip6_models)
kde = st.gaussian_kde(geoff_df.T)
geoff_sample = kde.resample(size=int(SAMPLES*1.25), seed = SEEDS[15])

# remove unphysical combinations
geoff_sample[:,geoff_sample[0,:] <= 0] = np.nan
geoff_sample[1, :] = st.truncnorm.rvs(-2, 2, loc=-4/3, scale=0.5, size=int(SAMPLES*1.25), random_state=SEEDS[16])
geoff_sample[:,geoff_sample[2,:] <= 0] = np.nan
geoff_sample[:,geoff_sample[3,:] <= 0] = np.nan
geoff_sample[:,geoff_sample[4,:] <= 0] = np.nan
geoff_sample[:,geoff_sample[5,:] <= 0] = np.nan

mask = np.all(np.isnan(geoff_sample), axis=0)
geoff_sample = geoff_sample[:,~mask][:,:SAMPLES]
geoff_sample_df=pd.DataFrame(data=geoff_sample.T, columns=['q4x','lamg','cmix','cdeep','gamma_2l','eff'])
geoff_sample_df.to_csv('../data_output/geoff_sample.csv')
geoff_sample_df

f2x = st.norm.rvs(loc=F2XCO2_MEAN, scale=F2XCO2_NINETY/NINETY_TO_ONESIGMA, size=SAMPLES, random_state=SEEDS[73])

ecs = -f2x/geoff_sample[1,:]
tcr = f2x/(-geoff_sample[1,:] + geoff_sample[4,:]*geoff_sample[5,:])

### Forcing uncertainties

In [9]:
# these are standard deviations of the scale factor for normally distributed forcings (mean = 1). The list below is expressed in terms of 5-95% ranges.
unc_ranges = np.array([
    0.12,      # CO2
    0.20,      # CH4: updated value from etminan 2016
    0.14,      # N2O
    0.19,      # other WMGHGs
    0.50,      # Total ozone
    1.00,      # stratospheric WV from CH4
    0.70,      # contrails approx - half-normal
    1.25,      # bc on snow - half-normal
    0.50,      # land use change
    5.0/20.0,  # volcanic
    0.50,      # solar (amplitude)
])/NINETY_TO_ONESIGMA

NORMALS = len(unc_ranges)

scale_normals = st.norm.rvs(
    size=(SAMPLES,NORMALS),
    loc=np.ones((SAMPLES,NORMALS)),
    scale=np.ones((SAMPLES, NORMALS)) * unc_ranges[None,:],
    random_state=SEEDS[4]
)

## bc snow is asymmetric Gaussian. We can just scale the half of the distribution above/below best estimate
scale_normals[scale_normals[:,7]<1,7] = 0.08/0.1*(scale_normals[scale_normals[:,7]<1,7]-1) + 1

## so is contrails - the benefits of doing this are tiny :)
scale_normals[scale_normals[:,6]<1,6] = 0.0384/0.0406*(scale_normals[scale_normals[:,6]<1,6]-1) + 1

trend_solar = st.norm.rvs(size=SAMPLES, loc=+0.01, scale=0.07/NINETY_TO_ONESIGMA, random_state=SEEDS[50])

In [10]:
with open("../data_input/cmip6_aerosol_coefficients.json") as json_file:
    cmip6_aerosol_data = json.load(json_file)

In [11]:
cmip6_aci = np.zeros((11, 2))
for i, model in enumerate(['CanESM5', 'E3SM', 'GFDL-ESM4', 'GFDL-CM4', 'GISS-E2-1-G', 'HadGEM3-GC31-LL', 'IPSL-CM6A-LR', 'MIROC6', 'MRI-ESM2-0', 'NorESM2-LM', 'UKESM1-0-LL']):
    for j, species in enumerate(['n0','n1']):
        cmip6_aci[i,j] = np.log(cmip6_aerosol_data[model]['ERFaci'][species])
kde = st.gaussian_kde(cmip6_aci.T)
aci_coeffs=np.exp(kde.resample(size=int(SAMPLES), seed=SEEDS[8]).T)

In [12]:
bc_20101750 = st.norm.rvs(loc=0.3, scale=0.2/NINETY_TO_ONESIGMA, size=SAMPLES, random_state=SEEDS[95])
oc_20101750 = st.norm.rvs(loc=-0.09, scale=0.07/NINETY_TO_ONESIGMA, size=SAMPLES, random_state=SEEDS[96])
so2_20101750 = st.norm.rvs(loc=-0.4, scale=0.2/NINETY_TO_ONESIGMA, size=SAMPLES, random_state=SEEDS[97])
nit_20101750 = st.norm.rvs(loc=-0.11, scale=0.05/NINETY_TO_ONESIGMA, size=SAMPLES, random_state=SEEDS[98])

In [13]:
# Get SSP historical emissions
ssp_df = pd.read_csv('../data_input_large/rcmip-emissions-annual-means-v5-1-0.csv')
species = [
    'Emissions|Sulfur',
    'Emissions|BC',
    'Emissions|OC',
    'Emissions|NH3',
    'Emissions|NOx'
]

unit_convert = np.ones(5)
unit_convert[0] = 32/64 # follow zeb exactly, but would be better to use fair.constants.molwt
unit_convert[4] = 14/46

emissions_out = np.zeros((351,5))

years_future = [2015] + list(range(2020,2101,10))
for i, specie in enumerate(species):
    emissions_out[:265,i] = ssp_df.loc[
        (ssp_df['Model']=='MESSAGE-GLOBIOM')&
        (ssp_df['Region']=='World')&
        (ssp_df['Scenario']=='ssp245')&
        (ssp_df['Variable']==specie),
        '1750':'2014']*unit_convert[i]
    f = interp1d(years_future, ssp_df.loc[
        (ssp_df['Model']=='MESSAGE-GLOBIOM')&
        (ssp_df['Region']=='World')&
        (ssp_df['Scenario']=='ssp245')&
        (ssp_df['Variable']==specie),'2015':'2100'
    ].dropna(axis=1))
    emissions_out[265:, i] = f(np.arange(2015, 2101))*unit_convert[i]